In [3]:
import pandas as pd

# 1 Data Cleaning

In [4]:
df_historical_data = pd.read_csv('../data/raw/world_cup_historical_data.csv')
df_fixture = pd.read_csv('../data/raw/world_cup_fixture.csv')
df_missing_data = pd.read_csv('../data/raw/fifa_worldcup_missing_data.csv')

### 1.1 Cleaning df_fixture

In [5]:
df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip()

### 1.2 Cleaning df_missing_data and adding it to df_historical_data

In [7]:
# null data
#df_missing_data[df_missing_data['home'].isnull()]

# drop null data
df_missing_data.dropna(inplace=True)

# concatenate data frames and clean
df_historical_data = pd.concat([df_historical_data, df_missing_data], ignore_index=True)
df_historical_data.drop_duplicates(inplace=True)
df_historical_data.sort_values('year', inplace=True)
df_historical_data

,home,score,away,year
0,France,4–1,Mexico,1930
1,Argentina,1–0,France,1930
2,Chile,3–0,Mexico,1930
3,Chile,1–0,France,1930
4,Argentina,6–3,Mexico,1930
...,...,...,...,...
921,Croatia,1–1 (a.e.t.),Brazil,2022
928,Argentina,3–3 (a.e.t.),France,2022
927,Croatia,2–1,Morocco,2022
926,France,2–0,Morocco,2022


### 1.3 Cleaning df_historical_data

In [10]:
# deleting match with walk over
df_historical_data[df_historical_data['home'].str.contains('Sweden') & 
                   df_historical_data['away'].str.contains('Austria')]

delete_index = df_historical_data[(df_historical_data['home'].str.contains('Sweden')) & 
                                  (df_historical_data['away'].str.contains('Austria'))].index

df_historical_data.drop(index=delete_index, inplace=True)

In [ ]:
# Checking the walk over was dropped
df_historical_data[df_historical_data['home'].str.contains('Sweden') & 
                   df_historical_data['away'].str.contains('Austria')]

,home,score,away,year


In [ ]:
# columns scores with not only digits and "-" --> [^ ]: Matches characteers not in brackets
df_historical_data[df_historical_data['score'].str.contains('[^\d–]')]

df_historical_data['score'] = df_historical_data['score'].str.replace('[^\d–]', '', regex=True)

<>:2: SyntaxWarning: "\d" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\d"? A raw string is also an option.
<>:4: SyntaxWarning: "\d" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\d"? A raw string is also an option.
<>:2: SyntaxWarning: "\d" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\d"? A raw string is also an option.
<>:4: SyntaxWarning: "\d" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\d"? A raw string is also an option.
C:\Users\Tyler\AppData\Local\Temp\ipykernel_22996\3455599414.py:2: SyntaxWarning: "\d" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\d"? A raw string is also an option.
  df_historical_data[df_historical_data['score'].str.contains('[^\d–]')]
C:\Users\Tyler\AppData\Local\Temp\ipykernel_22996\3455599414.py:4: SyntaxWarning: "\d" is an invalid 

In [21]:
# cleaning score and home/away columns
df_historical_data['home'] = df_historical_data['home'].str.strip()
df_historical_data['away'] = df_historical_data['away'].str.strip()

In [22]:
# splitting score columns into home and away goals
df_historical_data[['home_goals', 'away_goals']] = df_historical_data['score'].str.split('–', expand=True)

In [23]:
df_historical_data

,home,score,away,year,home_goals,away_goals
0,France,4–1,Mexico,1930,4,1
1,Argentina,1–0,France,1930,1,0
2,Chile,3–0,Mexico,1930,3,0
3,Chile,1–0,France,1930,1,0
4,Argentina,6–3,Mexico,1930,6,3
...,...,...,...,...,...,...
921,Croatia,1–1,Brazil,2022,1,1
928,Argentina,3–3,France,2022,3,3
927,Croatia,2–1,Morocco,2022,2,1
926,France,2–0,Morocco,2022,2,0


In [ ]:
# dropping score column after splitting into home and away goals
df_historical_data.drop('score', axis=1, inplace=True)

KeyError: "['score'] not found in axis"

In [26]:
df_historical_data

,home,away,year,home_goals,away_goals
0,France,Mexico,1930,4,1
1,Argentina,France,1930,1,0
2,Chile,Mexico,1930,3,0
3,Chile,France,1930,1,0
4,Argentina,Mexico,1930,6,3
...,...,...,...,...,...
921,Croatia,Brazil,2022,1,1
928,Argentina,France,2022,3,3
927,Croatia,Morocco,2022,2,1
926,France,Morocco,2022,2,0


In [28]:
# renaming columns and changing data types
df_historical_data.rename(columns={'year': 'Year', 
                                   'home': 'HomeTeam', 
                                   'away': 'AwayTeam', 
                                   'home_goals': 'HomeGoals', 
                                   'away_goals': 'AwayGoals'}, inplace=True)

df_historical_data = df_historical_data.astype({'AwayGoals': 'int',
                                                'HomeGoals': 'int',
                                                'Year': 'int'})

In [29]:
# Creating new column "totalgoals"
df_historical_data['TotalGoals'] = df_historical_data['HomeGoals'] + df_historical_data['AwayGoals']

In [30]:
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
1,Argentina,France,1930,1,0,1
2,Chile,Mexico,1930,3,0,3
3,Chile,France,1930,1,0,1
4,Argentina,Mexico,1930,6,3,9
...,...,...,...,...,...,...
921,Croatia,Brazil,2022,1,1,2
928,Argentina,France,2022,3,3,6
927,Croatia,Morocco,2022,2,1,3
926,France,Morocco,2022,2,0,2


## 2 Exporting clean dataframes

In [31]:
df_historical_data.to_csv('../data/processed/fifa_worldcup_matches_cleaned.csv', index=False)
df_fixture.to_csv('../data/processed/fifa_worldcup_fixtures_cleaned.csv', index=False)

## 3 Extra verifications

In [32]:
# verifiy number of matches per competition
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 
         1966, 1970, 1974, 1978, 1982, 1986, 1990, 1994,
         1998, 2002, 2006, 2010, 2014, 2018, 2022]

for year in years:
    print(year, len(df_historical_data[df_historical_data['Year'] == year]))

1930 18
1934 17
1938 18
1950 22
1954 26
1958 35
1962 32
1966 32
1970 32
1974 38
1978 38
1982 52
1986 52
1990 52
1994 52
1998 64
2002 64
2006 64
2010 64
2014 64
2018 64
2022 64
